In [ ]:
import pandas as pd
import os
from tqdm.notebook import tqdm


In [ ]:
def filter_df(df):
    sdf = df.loc[df.algorithm == "Average", ['seed', 'experiment', 'algorithm']].groupby(
        by=['seed', 'experiment']).count().reset_index()
    seeds = sdf.seed.unique()

    from ast import literal_eval

    from collections import Counter
    import numpy as np
    col_defaults = {}
    for col in tqdm(df.columns[1:]):
        if col in ('Unnamed: 0', 'experiment', 'average reward', "variance", 'cum rewar', 'cum reward', 't', "algorithm", "covariance"):
            continue
        counts = df[df.algorithm == "Average"][col].value_counts()
        col_defaults[col] = sorted(
            [k for k in counts.index if col == 'desired_var' or counts[k] == np.max(list(counts.values))])

    col_defaults['n_clusters'] = [1, 2]

    def alg_to_method(s):
        # if s=='EXPL3Ensemble_0.1_1':
        #     return 'Meta-CMAB (UCB)'
        # if s=='Falcon17.1085414':
        #     return 'Meta-CMAB (FALCON)'
        # if s=='Meta-CMAB_adaptiveTS':
        #     return 'rem'#Meta-CMAB (TS)'
        # if s=='Meta-CMAB_0':
        #     return 'Greedy'

        # # if s=='Cover_0.01_0.05__F_2_N':
        # if s=='Cover_0.01_0.05__F_2_N':
        #     return 'Meta-CMAB (ILTCB)'
        # if s=='Meta-CMAB_1':
        #     return 'Meta-CMAB (LinUCB)'
        # if s=='SquareCB':
        #     return 'Meta-CMAB (SquareCB)'
        if s == 'best':
            return 'Best Expert'
        if s == 'worst':
            return 'Worst Expert'
        return s
    print(df.columns)
    gdf = df.groupby(['bernoulli', 't', 'algorithm',
                      'advice type', 'K', 'N', '(K,N)', 'configuration',
                      'spread_variance', 'n_clusters', 'desired_var', 'desired_covar',
                      'problem', 'n_trials', 'alpha']).mean().reset_index()
    gdf['Method'] = gdf['algorithm'].map(alg_to_method)
    gdf['Correlation'] = gdf['desired_covar'].map(
        lambda s: {0.1: 'Low', 0.5: 'Medium', 0.9: 'High', 2: 'Random'}[s])
    col_defaults['Correlation'] = list(map(lambda s: {
                                       0.1: 'Low', 0.5: 'Medium', 0.9: 'High', 2: 'Random'}[s], col_defaults['desired_covar']))
    gdf['ϵ'] = gdf['desired_var']
    col_defaults['ϵ'] = col_defaults['desired_var']
    # gdf=gdf[gdf['problem']==10]

    return col_defaults, gdf


In [ ]:
from p_tqdm import p_umap
import h5py
import h5py
all_dfs = []
base = 'F:/res/results_tmp_old/'
base = 'results_tmp/'


def read_file(filename):
    group = '/scratch/brussel/103/vsc10321/results_tmp/'
    group = 'results_tmp/'
    import pandas as pd
    import h5py
    if 'bak' not in filename:
        return None
    try:
        with h5py.File(filename, 'r') as f:

            return (pd.concat((pd.DataFrame(f[group+k][:]) for k in (f[group].keys()))))
    except:
        print("failed ", group)
        return None


results = p_umap(read_file, [
                 base+filename for filename in list(os.listdir(base))][:], smoothing=0)


In [ ]:
df = pd.concat([r for r in results if r is not None])
df['algorithm'] = df['algorithm'].str.decode("utf-8")
df['problem'] = df['problem'].str.decode("utf-8")
col_defaults, gdf = filter_df(df)


In [ ]:
sdf = df.groupby(['bernoulli', 't', 'algorithm',
                  'advice type', 'K', 'N', '(K,N)', 'configuration',
                  'spread_variance', 'n_clusters', 'desired_var', 'desired_covar',
                  'problem', 'n_trials', 'alpha']).std().reset_index()


In [ ]:
alg_mask = (
    (gdf.Method != 'Linear Optimum') 
)
alg_mask = True
meta_style = (1, 0)
linestyles = {'Meta-CMAB (Cover)': (1, 0), 'Linear Optimum': (1, 1), 'Meta-CMAB (ILTCB)': meta_style, 'Meta-CMAB (FALCON)': meta_style,
              'Average': (3, 1, 1, 1), 'EXP4-IX': (5, 1), 'Meta-CMAB (UCB)': meta_style,
              'Meta-MAB': (5, 1), 'Best Expert': (1, 1), 'Worst Expert': (1, 1)}

palette = {'Average': 'C1', 'EXP4-IX': 'C0',
           'Linear Optimum': 'C9',
           'Weighted Average Optimum': 'C8',
           'Meta-CMAB (ILTCB)': 'C4',
           'Meta-CMAB (FALCON)': 'C6',
           'Meta-CMAB (UCB)': 'C2',
           'Best Expert': 'C5', 'Worst Expert': 'C6',
           'Meta-MAB': 'C3', 'optimal': 'grey', 'random': 'red',
           }


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_context("poster")
row = None
col = 'Correlation' 
style = None
row = None
x_axis = 'ϵ'
yaxis = 'average reward'
ALPHA = 1

for problem in ('regression','classification'):
    for spread_variance in (False, True):
        print(problem, spread_variance)
        plotdf = gdf[(True)
                     & alg_mask
                     & (gdf['n_trials'].isin([100]))
                     & (gdf['problem'].isin([problem]))
                     & (gdf['spread_variance'].isin([spread_variance]))
                     & (~gdf.algorithm.str.contains("expert"))
                     ]
        col_values = ["Low", "Medium", "High"]

        methods = [v for v in plotdf.Method.unique() if v not in ("optimal", "random")]
        methods = sorted(methods, key=lambda s: ('Meta-CMAB' not in s, 'Expert' in s, s))
        g = sns.relplot(data=plotdf[(~plotdf.algorithm.str.contains("optimal"))
                                    & (~plotdf.algorithm.str.contains("random"))],
                        y=yaxis, x=x_axis, hue="Method", col_order=col_values, hue_order=methods, palette=palette, ci=None, kind='line', legend=False, style='Method', dashes=linestyles, row=row, col=col)

        labels = "ABCDEFGH"
        label_idx = 0
        for ax_rows in g.axes:
            for ax, col_name in tqdm(zip(ax_rows, col_values), total=len(col_values)):

                random_values = plotdf[(plotdf[col] == col_name) & (plotdf.algorithm.str.contains(
                    "optimal"))].groupby(x_axis).mean().reset_index()[[x_axis, "average reward"]]
                ax.plot(random_values[x_axis].values, random_values['average reward'].values,
                        zorder=0, c='grey', linestyle='--', linewidth=1)
                random_values = plotdf[(plotdf[col] == col_name) & (plotdf.algorithm.str.contains(
                    "random"))].groupby(x_axis).mean().reset_index()[[x_axis, "average reward"]]
                ax.plot(random_values[x_axis].values, random_values['average reward'].values,
                        zorder=0, c='red', linestyle='--', linewidth=1)
                ax.set_title(labels[label_idx].upper() +
                             "                              ")
                label_idx += 1

        h = plt.gca().get_lines()
        h2 = []
        from matplotlib.lines import Line2D
        for k in methods:
            h2.append(Line2D([0], [0], color=palette[k],
                      dashes=linestyles[k], label=k))
        h = h2
        new_hues = list(methods)
        

        lg = ax.legend(loc='upper center', bbox_to_anchor=(-.8, 1.45+(.15 if True else .25)),
                       fancybox=True, shadow=True, ncol=3, handles=h[:], labels=new_hues, handlelength=2.8, frameon=False)
    
        plt.ylim(0, 1)
        os.makedirs('figures', exist_ok=True)
        plt.savefig(f'figures/{"regr" if problem=="regression" else "class"}_correlation_error{"_het" if spread_variance else "_hom"}.pdf',
                    bbox_extra_artists=(lg,), bbox_inches='tight')
        plt.show()


In [ ]:
from matplotlib.lines import Line2D
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_context("poster")
row = None
col = 'Correlation'  # None# 'n_clusters'
style = 'correlated errors'
row = 'bernoulli'
x_axis = 'ϵ'
yaxis = 'average reward'
ALPHA = 1

plotdf = gdf[(True)
             & alg_mask
             & (gdf['n_trials'].isin([100]))
             & (~gdf.algorithm.str.contains("expert"))
             ]
col_values = ["Low", "Medium", "High"]

methods = [v for v in plotdf.Method.unique() if v not in ("optimal", "random")]
methods = sorted(methods, key=lambda s: (
    'Meta-CMAB' not in s, 'Expert' in s, s))
g = sns.relplot(data=plotdf[(~plotdf.algorithm.str.contains("optimal"))
                            & (~plotdf.algorithm.str.contains("random"))],
                y=yaxis, x=x_axis, hue="Method", col_order=col_values, hue_order=methods, palette=palette, ci=None, kind='line', legend=False, style='Method', dashes=linestyles, row=row, col=col)

labels = "ABCDEFGH"
label_idx = 0
for ax_rows in g.axes:
    for ax, col_name in tqdm(zip(ax_rows, col_values), total=len(col_values)):

        random_values = plotdf[(plotdf[col] == col_name) & (plotdf.algorithm.str.contains(
            "optimal"))].groupby(x_axis).mean().reset_index()[[x_axis, "average reward"]]
        ax.plot(random_values[x_axis].values, random_values['average reward'].values,
                zorder=0, c='grey', linestyle='--', linewidth=1)
        random_values = plotdf[(plotdf[col] == col_name) & (plotdf.algorithm.str.contains(
            "random"))].groupby(x_axis).mean().reset_index()[[x_axis, "average reward"]]
        ax.plot(random_values[x_axis].values, random_values['average reward'].values,
                zorder=0, c='red', linestyle='--', linewidth=1)
        ax.set_title(labels[label_idx].upper() +
                     "                              ")
        label_idx += 1

h = plt.gca().get_lines()
h2 = []
for k in methods:
    h2.append(Line2D([0], [0], color=palette[k],
              dashes=linestyles[k], label=k))
h = h2  
new_hues = list(methods)


lg = ax.legend(loc='upper center', bbox_to_anchor=(-.8, 1.45+(.15 if True else .25)),
               fancybox=True, shadow=True, ncol=3, handles=h[:], labels=new_hues, handlelength=2.8, frameon=False)

plt.ylim(0, 1)
os.makedirs('figures', exist_ok=True)
plt.savefig(f'figures/correlation_error.pdf',
            bbox_extra_artists=(lg,), bbox_inches='tight')
plt.show()


### PLOT BY N TRIALS


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_context("poster")
row = None  # 'Correlation'
col = 'Correlation'  # None# 'n_clusters'
col = 'n_trials'  # None# 'n_clusters'
style = 'correlated errors'
style = None
x_axis = 'ϵ'
yaxis = 'average reward'
ALPHA = 1
for problem in ('regression','classification'):

    for spread_variance in (False, True):
        print(problem, spread_variance)
        plotdf = gdf[(True)
                     & alg_mask
                     & (gdf['problem'] == problem)
                     & (gdf['spread_variance'].isin([spread_variance]))
                     & (~gdf.algorithm.str.contains("expert"))
                     ]
        col_values = [t for t in [10, 100, 1000, 10000]
                      if t in gdf.n_trials.unique()]

        methods = [v for v in plotdf.Method.unique(
        ) if v not in ("optimal", "random")]
        methods = sorted(methods, key=lambda s: (
            'Meta-CMAB' not in s, 'Expert' in s, s))
        g = sns.relplot(data=plotdf[(~plotdf.algorithm.str.contains("optimal"))
                                    & (~plotdf.algorithm.str.contains("random"))],
                        y=yaxis, x=x_axis, col_order=col_values, hue="Method", hue_order=methods, palette=palette, ci=None, kind='line', legend=False, style='Method', dashes=linestyles, row=row, col=col)

        labels = "ABCDEFGHIJKLMNOPQRST"
        label_idx = 0
        for ax_rows in g.axes:
            for ax, col_name in tqdm(zip(ax_rows, sorted(plotdf[col].unique())), total=len(sorted(plotdf[col].unique()))):

                random_values = plotdf[(plotdf[col] == col_name) & (plotdf.algorithm.str.contains(
                    "optimal"))].groupby(x_axis).mean().reset_index()[[x_axis, "average reward"]]
                ax.plot(random_values[x_axis].values, random_values['average reward'].values,
                        zorder=0, c='grey', linestyle='--', linewidth=1)
                random_values = plotdf[(plotdf[col] == col_name) & (plotdf.algorithm.str.contains(
                    "random"))].groupby(x_axis).mean().reset_index()[[x_axis, "average reward"]]
                ax.plot(random_values[x_axis].values, random_values['average reward'].values,
                        zorder=0, c='red', linestyle='--', linewidth=1)
                ax.set_title(labels[label_idx].upper() +
                             "                              ")
                label_idx += 1

        h = plt.gca().get_lines()
        h2 = []
        from matplotlib.lines import Line2D
        for k in methods:
            h2.append(Line2D([0], [0], color=palette[k],
                      dashes=linestyles[k], label=k))
        h = h2 
        new_hues = list(methods)
        if len(plotdf) > 0:
            lg = ax.legend(loc='upper center', bbox_to_anchor=(-1.2, 1.45+(.15 if True else .25)),
                           fancybox=True, shadow=True, ncol=3, handles=h[:], labels=new_hues, handlelength=2.8, frameon=False)

            plt.ylim(0, 1)
            
            os.makedirs('figures', exist_ok=True)
            plt.savefig(f'figures/{"class_" if problem=="classification" else "regr_"}trials_errors{"_het" if spread_variance else "_hom"}.pdf',
                        bbox_extra_artists=(lg,), bbox_inches='tight')

        plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_context("poster")
row = None  
col = 'Correlation' 
col = 'ϵ'  
style = 'correlated errors'
style = None
x_axis = 'T'
yaxis = 'average reward'
ALPHA = 1
gdf["T"] = gdf["n_trials"]
for problem in ('regression','classification'):

    for spread_variance in (False, True):
        print(problem, spread_variance)
        plotdf = gdf[(True)
                     & alg_mask
                     & (gdf['problem'] == problem)
                     & (gdf['spread_variance'].isin([spread_variance]))
                     & (~gdf.algorithm.str.contains("expert"))
                     ]

        methods = [v for v in plotdf.Method.unique(
        ) if v not in ("optimal", "random")]
        methods = sorted(methods, key=lambda s: (
            'Meta-CMAB' not in s, 'Expert' in s, s))
        g = sns.relplot(data=plotdf[(~plotdf.algorithm.str.contains("optimal"))
                                    & (~plotdf.algorithm.str.contains("random"))],
                        y=yaxis, x=x_axis, hue="Method", hue_order=methods, palette=palette, ci=None, kind='line', legend=False, style='Method', dashes=linestyles, row=row, col=col)

        labels = "ABCDEFGHIJKLMNOPQRST"
        label_idx = 0
        for ax_rows in g.axes:
            for ax, col_name in tqdm(zip(ax_rows, sorted(plotdf[col].unique())), total=len(sorted(plotdf[col].unique()))):

                random_values = plotdf[(plotdf[col] == col_name) & (plotdf.algorithm.str.contains(
                    "optimal"))].groupby(x_axis).mean().reset_index()[[x_axis, "average reward"]]
                ax.plot(random_values[x_axis].values, random_values['average reward'].values,
                        zorder=0, c='grey', linestyle='--', linewidth=1)
                random_values = plotdf[(plotdf[col] == col_name) & (plotdf.algorithm.str.contains(
                    "random"))].groupby(x_axis).mean().reset_index()[[x_axis, "average reward"]]
                ax.plot(random_values[x_axis].values, random_values['average reward'].values,
                        zorder=0, c='red', linestyle='--', linewidth=1)
                ax.set_title(labels[label_idx].upper() +
                             "                              ")
                label_idx += 1

        h = plt.gca().get_lines()
        h2 = []
        from matplotlib.lines import Line2D
        for k in methods:
            h2.append(Line2D([0], [0], color=palette[k],
                      dashes=linestyles[k], label=k))
        h = h2 
        new_hues = list(methods)
        if len(plotdf) > 0:
           
            lg = ax.legend(loc='upper center', bbox_to_anchor=(-2, 1.45+(.15 if True else .25)),
                           fancybox=True, shadow=True, ncol=4, handles=h[:], labels=new_hues, handlelength=2.8, frameon=False)

            plt.ylim(0, 1)
            plt.xscale('log')
            os.makedirs('figures', exist_ok=True)
            plt.savefig(f'figures/{"class_" if problem=="classification" else "regr_"}trials_errors{"_het" if spread_variance else "_hom"}.pdf',
                        bbox_extra_artists=(lg,), bbox_inches='tight')

        plt.show()


In [ ]:
from curlyBrace import curlyBrace
from matplotlib.lines import Line2D
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_context("poster")
row = 'problem'
# col = 'Correlation' #None# 'n_clusters'
col = 'spread_variance'  # None# 'n_clusters'
style = 'correlated errors'
style = None
x_axis = 'T'
yaxis = 'average reward'
ALPHA = 1
if "T" not in gdf.columns:
    gdf["T"] = gdf["n_trials"]

plotdf = gdf[(True)
             & alg_mask
             & (~gdf.algorithm.str.contains("expert"))
             ]
row_order = sorted(plotdf[row].unique(), reverse=True)
methods = [v for v in plotdf.Method.unique() if v not in ("optimal", "random")]
methods = sorted(methods, key=lambda s: (
    'Meta-CMAB' not in s, 'Expert' in s, s))
g = sns.relplot(data=plotdf[(~plotdf.algorithm.str.contains("optimal"))
                            & (~plotdf.algorithm.str.contains("random"))],
                y=yaxis, x=x_axis, hue="Method", hue_order=methods, row_order=row_order, palette=palette, ci=None, kind='line', legend=False, style='Method', dashes=linestyles, row=row, col=col)

Ks = sorted(plotdf.K.unique())
labels = "ABCDEFGH"
label_idx = 0
for ax_rows, row_name in zip(g.axes, row_order):
    for ax, col_name in tqdm(zip(ax_rows, sorted(plotdf[col].unique())), total=len(sorted(plotdf[col].unique()))):

        random_values = plotdf[(plotdf[col] == col_name) & (plotdf[row] == row_name) & (
            plotdf.algorithm.str.contains("optimal"))].groupby(x_axis).mean().reset_index()[[x_axis, "average reward"]]
        ax.plot(random_values[x_axis].values, random_values['average reward'].values,
                zorder=0, c='grey', linestyle='--', linewidth=1)
        random_values = plotdf[(plotdf[col] == col_name) & (plotdf[row] == row_name) & (
            plotdf.algorithm.str.contains("random"))].groupby(x_axis).mean().reset_index()[[x_axis, "average reward"]]
        ax.plot(random_values[x_axis].values, random_values['average reward'].values,
                zorder=0, c='red', linestyle='--', linewidth=1)
        ax.set_title(labels[label_idx].upper() +
                     "                              ")
        label_idx += 1

h = plt.gca().get_lines()
h2 = []
for k in methods:
    h2.append(Line2D([0], [0], color=palette[k],
              dashes=linestyles[k], label=k))
h = h2 
new_hues = list(methods)

lg = ax.legend(loc='upper center', bbox_to_anchor=(-.2, 2.65+(.15 if True else .25)),
               fancybox=True, shadow=True, ncol=3, handles=h[:], labels=new_hues, handlelength=2.8, frameon=False)

plt.ylim(0, 1)
os.makedirs('figures', exist_ok=True)


k_r1 = 0.05

font = {'family': 'serif',
        'color':  'k',
        'weight': 'bold',
        'size': 20,
        }

plt.xscale("log")
ax2 = plt.axes([0, -0.1, 1.1, 1.1], facecolor=(1, 1, 1, 0))
ax2.set_xlim(0, 1)
ax2.set_ylim(0, 1)
curlyBrace(plt.gcf(), ax2,  [0.905, 0.55], [0.905, 0.15], k_r1, bool_auto=False,
           str_text="regression", color='r', lw=2, int_line_num=1, fontdict=font)

curlyBrace(plt.gcf(), ax2,   [0.905, .98], [0.905, .56], k_r1, bool_auto=False,
           str_text="classification", color='r', lw=2, int_line_num=1, fontdict=font)

curlyBrace(plt.gcf(), ax2,  [0.48, 0.095], [0.09, 0.095], k_r1, bool_auto=False,
           str_text="homogeneous", color='r', lw=2, int_line_num=1, fontdict=font)

curlyBrace(plt.gcf(), ax2,   [.9, 0.095], [0.49, .095], k_r1, bool_auto=False,
           str_text="heterogeneous", color='r', lw=2, int_line_num=1, fontdict=font)

ax2.spines['right'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax2.spines['left'].set_visible(False)
ax2.spines['bottom'].set_visible(False)
ax2.set_yticklabels([])
ax2.set_xticklabels([])
ax2.set_xticks([])
ax2.set_yticks([])
plt.savefig(f'figures/trials.pdf',
            bbox_extra_artists=(lg,), bbox_inches='tight')
plt.show()


In [ ]:
from curlyBrace import curlyBrace
from matplotlib.lines import Line2D
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_context("poster")
row = None  
col = 'problem'  
style = 'correlated errors'
style = None
x_axis = 'T'
yaxis = 'average reward'
ALPHA = 1
if "T" not in gdf.columns:
    gdf["T"] = gdf["n_trials"]

plotdf = gdf[(True)
             & alg_mask
             & (~gdf.algorithm.str.contains("expert"))
             ]
col_order = sorted(plotdf[col].unique(), reverse=True)
methods = [v for v in plotdf.Method.unique() if v not in ("optimal", "random")]
methods = sorted(methods, key=lambda s: (
    'Meta-CMAB' not in s, 'Expert' in s, s))
g = sns.relplot(data=plotdf[(~plotdf.algorithm.str.contains("optimal"))
                            & (~plotdf.algorithm.str.contains("random"))],
                y=yaxis, x=x_axis, hue="Method", col_order=col_order, hue_order=methods, palette=palette, ci=None, kind='line', legend=False, style='Method', dashes=linestyles, row=row, col=col)

Ks = sorted(plotdf.K.unique())
labels = "ABCDEFGH"
label_idx = 0
for ax_rows in g.axes:
    for ax, col_name in tqdm(zip(ax_rows, col_order), total=len(sorted(plotdf[col].unique()))):

        random_values = plotdf[(plotdf[col] == col_name) & (plotdf.algorithm.str.contains(
            "optimal"))].groupby(x_axis).mean().reset_index()[[x_axis, "average reward"]]
        ax.plot(random_values[x_axis].values, random_values['average reward'].values,
                zorder=0, c='grey', linestyle='--', linewidth=1)
        random_values = plotdf[(plotdf[col] == col_name) & (plotdf.algorithm.str.contains(
            "random"))].groupby(x_axis).mean().reset_index()[[x_axis, "average reward"]]
        ax.plot(random_values[x_axis].values, random_values['average reward'].values,
                zorder=0, c='red', linestyle='--', linewidth=1)
        ax.set_title(labels[label_idx].upper() +
                     "                              ")
        label_idx += 1

h = plt.gca().get_lines()
h2 = []
for k in methods:
    h2.append(Line2D([0], [0], color=palette[k],
              dashes=linestyles[k], label=k))
h = h2  
new_hues = list(methods)

lg = ax.legend(loc='upper center', bbox_to_anchor=(-.2, 1.45+(.15 if True else .25)),
               fancybox=True, shadow=True, ncol=3, handles=h[:], labels=new_hues, handlelength=2.8, frameon=False)

plt.ylim(0, 1)
os.makedirs('figures', exist_ok=True)


# coefficient for curly bracket 1
k_r1 = 0.05

font = {'family': 'serif',
        'color':  'k',
        'weight': 'bold',
        'size': 20,
        }


plt.xscale("log")

plt.savefig(f'figures/trials_aggr.pdf',
            bbox_extra_artists=(lg,), bbox_inches='tight')
plt.show()


### PLOT BY ARMS


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_context("poster")
row = None  
col = 'spread_variance' 
style = 'correlated errors'
style = None
x_axis = 'K'
yaxis = 'average reward'
ALPHA = 1


for problem in ('regression', 'classification'):

    plotdf = gdf[(True)
                 & alg_mask
                 & ((~gdf.Method.str.contains('Expert')))
                 & (gdf.N == 16)
                 & (gdf['n_trials'].isin([100]))
                 & (gdf['bernoulli'].isin([True]))
                 & (gdf['desired_covar'] != 2)
                 & (gdf['problem'] == problem)
                 & (gdf['n_clusters'].isin([2]))
                 & (~gdf.algorithm.str.contains("expert"))

                 ]

    methods = [v for v in plotdf.Method.unique(
    ) if v not in ("optimal", "random")]
    methods = sorted(methods, key=lambda s: (
        'Meta-CMAB' not in s, 'Expert' in s, s))
    g = sns.relplot(data=plotdf[(~plotdf.algorithm.str.contains("optimal"))
                                & (~plotdf.algorithm.str.contains("random"))],
                    y=yaxis, x=x_axis, hue="Method", hue_order=methods, palette=palette, ci=None, kind='line', legend=False, style='Method', dashes=linestyles, row=row, col=col)


    Ks = sorted(plotdf.K.unique())
    labels = "ABCDEFGH"
    label_idx = 0
    g.set(xscale='log')
    g.set(xticks=Ks, xticklabels=list(map(str, Ks)))
    for ax_rows in g.axes:
        for ax, col_name in tqdm(zip(ax_rows, sorted(plotdf[col].unique())), total=len(sorted(plotdf[col].unique()))):

            random_values = plotdf[(plotdf[col] == col_name) & (plotdf.algorithm.str.contains(
                "optimal"))].groupby(x_axis).mean().reset_index()[[x_axis, "average reward"]]
            ax.plot(random_values[x_axis].values, random_values['average reward'].values,
                    zorder=0, c='grey', linestyle='--', linewidth=1)
            random_values = plotdf[(plotdf[col] == col_name) & (plotdf.algorithm.str.contains(
                "random"))].groupby(x_axis).mean().reset_index()[[x_axis, "average reward"]]
            ax.plot(random_values[x_axis].values, random_values['average reward'].values,
                    zorder=0, c='red', linestyle='--', linewidth=1)
            ax.set_title(labels[label_idx].upper() +
                         "                              ")
            label_idx += 1
            ax.set_xticks([], minor=True)

    h = plt.gca().get_lines()
    h2 = []
    from matplotlib.lines import Line2D
    for k in methods:
        h2.append(Line2D([0], [0], color=palette[k],
                  dashes=linestyles[k], label=k))
    h = h2  
    new_hues = list(methods)

    lg = ax.legend(loc='upper center', bbox_to_anchor=(-.2, 1.55+(.15 if True else .25)),
                   fancybox=True, shadow=True, ncol=2, handles=h[:], labels=new_hues, handlelength=2.8, frameon=False)

    plt.ylim(0, 1)
    os.makedirs('figures', exist_ok=True)
    plt.savefig(f'figures/{"class_" if problem=="classification" else "regr_"}config_K.pdf',
                bbox_extra_artists=(lg,), bbox_inches='tight')


In [ ]:
from matplotlib.lines import Line2D
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_context("poster")
row = None 
col = 'problem'
style = 'correlated errors'
style = None
x_axis = 'K'
yaxis = 'average reward'
ALPHA = 1


plotdf = gdf[(True)
             & alg_mask
             & ((~gdf.Method.str.contains('Expert')))
             & (gdf.N == 16)
             & (gdf['n_trials'].isin([100]))
             & (~gdf.algorithm.str.contains("expert"))
             ]

methods = [v for v in plotdf.Method.unique() if v not in ("optimal", "random")]
methods = sorted(methods, key=lambda s: (
    'Meta-CMAB' not in s, 'Expert' in s, s))
g = sns.relplot(data=plotdf[(~plotdf.algorithm.str.contains("optimal"))
                            & (~plotdf.algorithm.str.contains("random"))],
                y=yaxis, x=x_axis, hue="Method", hue_order=methods, palette=palette, ci=None, kind='line', legend=False, style='Method', dashes=linestyles, row=row, col=col)

Ks = sorted(plotdf.K.unique())
labels = "CDEFGH"
label_idx = 0
g.set(xscale='log')
g.set(xticks=Ks, xticklabels=list(map(str, Ks)))
for ax_rows in g.axes:
    for ax, col_name in tqdm(zip(ax_rows, sorted(plotdf[col].unique())), total=len(sorted(plotdf[col].unique()))):

        random_values = plotdf[(plotdf[col] == col_name) & (plotdf.algorithm.str.contains(
            "optimal"))].groupby(x_axis).mean().reset_index()[[x_axis, "average reward"]]
        ax.plot(random_values[x_axis].values, random_values['average reward'].values,
                zorder=0, c='grey', linestyle='--', linewidth=1)
        random_values = plotdf[(plotdf[col] == col_name) & (plotdf.algorithm.str.contains(
            "random"))].groupby(x_axis).mean().reset_index()[[x_axis, "average reward"]]
        ax.plot(random_values[x_axis].values, random_values['average reward'].values,
                zorder=0, c='red', linestyle='--', linewidth=1)
        ax.set_title(labels[label_idx].upper() +
                     "                              ")
        label_idx += 1
        ax.set_xticks([], minor=True)

h = plt.gca().get_lines()
h2 = []
for k in methods:
    h2.append(Line2D([0], [0], color=palette[k],
              dashes=linestyles[k], label=k))
h = h2  
new_hues = list(methods)

lg = ax.legend(loc='upper center', bbox_to_anchor=(-.2, 1.55+(.15 if True else .25)),
               fancybox=True, shadow=True, ncol=2, handles=h[:], labels=new_hues, handlelength=2.8, frameon=False)

plt.ylim(0, 1)
os.makedirs('figures', exist_ok=True)
plt.savefig(f'figures/config_K.pdf',
            bbox_extra_artists=(lg,), bbox_inches='tight')


### PLOT BY EXPERTS


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_context("poster")
row = None 

col = 'spread_variance' 
style = 'correlated errors'
style = None
x_axis = 'N'
yaxis = 'average reward'
ALPHA = 1


for problem in ('regression', "classification"):

    plotdf = gdf[(True)
                 & alg_mask
                 & ((~gdf.Method.str.contains('Expert')))
                 & (gdf.K == 16)
                 & (gdf['n_trials'].isin([100]))
                 & (gdf['problem'] == problem)
                 & (~gdf.algorithm.str.contains("expert"))
                 ]

    methods = [v for v in plotdf.Method.unique(
    ) if v not in ("optimal", "random")]
    methods = sorted(methods, key=lambda s: (
        'Meta-CMAB' not in s, 'Expert' in s, s))
    g = sns.relplot(data=plotdf[(~plotdf.algorithm.str.contains("optimal"))
                                & (~plotdf.algorithm.str.contains("random"))],
                    y=yaxis, x=x_axis, hue="Method", hue_order=methods, palette=palette, ci=None, kind='line', legend=False, style='Method', dashes=linestyles, row=row, col=col)

    Ns = sorted(plotdf.N.unique())
    labels = "ABCDEFGH"
    label_idx = 0
    g.set(xscale='log')
    g.set(xticks=Ns, xticklabels=list(map(str, Ns)))
    for ax_rows in g.axes:
        for ax, col_name in tqdm(zip(ax_rows, sorted(plotdf[col].unique())), total=len(sorted(plotdf[col].unique()))):

            random_values = plotdf[(plotdf[col] == col_name) & (plotdf.algorithm.str.contains(
                "optimal"))].groupby(x_axis).mean().reset_index()[[x_axis, "average reward"]]
            ax.plot(random_values[x_axis].values, random_values['average reward'].values,
                    zorder=0, c='grey', linestyle='--', linewidth=1)
            random_values = plotdf[(plotdf[col] == col_name) & (plotdf.algorithm.str.contains(
                "random"))].groupby(x_axis).mean().reset_index()[[x_axis, "average reward"]]
            ax.plot(random_values[x_axis].values, random_values['average reward'].values,
                    zorder=0, c='red', linestyle='--', linewidth=1)
            ax.set_title(labels[label_idx].upper() +
                         "                              ")
            label_idx += 1
            ax.set_xticks([], minor=True)

    h = plt.gca().get_lines()
    h2 = []
    from matplotlib.lines import Line2D
    for k in methods:
        h2.append(Line2D([0], [0], color=palette[k],
                  dashes=linestyles[k], label=k))
    h = h2  
    new_hues = list(methods)

    lg = ax.legend(loc='upper center', bbox_to_anchor=(-.2, 1.55+(.15 if True else .25)),
                   fancybox=True, shadow=True, ncol=2, handles=h[:], labels=new_hues, handlelength=2.8, frameon=False)

    plt.ylim(0, 1)
    os.makedirs('figures', exist_ok=True)
    plt.savefig(f'figures/{"class_" if problem=="classification" else "regr_"}config_N.pdf',
                bbox_extra_artists=(lg,), bbox_inches='tight')


In [ ]:
from matplotlib.lines import Line2D
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_context("poster")
row = None 
col = 'problem' 
style = 'correlated errors'
style = None
x_axis = 'N'
yaxis = 'average reward'
ALPHA = 1


plotdf = gdf[(True)
             & alg_mask
             & ((~gdf.Method.str.contains('Expert')))
             & (gdf.K == 16)
             & (gdf['n_trials'].isin([100]))
             & (~gdf.algorithm.str.contains("expert"))
             ]

methods = [v for v in plotdf.Method.unique() if v not in ("optimal", "random")]
methods = sorted(methods, key=lambda s: (
    'Meta-CMAB' not in s, 'Expert' in s, s))
g = sns.relplot(data=plotdf[(~plotdf.algorithm.str.contains("optimal"))
                            & (~plotdf.algorithm.str.contains("random"))],
                y=yaxis, x=x_axis, hue="Method", hue_order=methods, palette=palette, ci=None, kind='line', legend=False, style='Method', dashes=linestyles, row=row, col=col)

Ns = sorted(plotdf.N.unique())
labels = "ABCDEFGH"
label_idx = 0
g.set(xscale='log')
g.set(xticks=Ns, xticklabels=list(map(str, Ns)))
for ax_rows in g.axes:
    for ax, col_name in tqdm(zip(ax_rows, sorted(plotdf[col].unique())), total=len(sorted(plotdf[col].unique()))):

        random_values = plotdf[(plotdf[col] == col_name) & (plotdf.algorithm.str.contains(
            "optimal"))].groupby(x_axis).mean().reset_index()[[x_axis, "average reward"]]
        ax.plot(random_values[x_axis].values, random_values['average reward'].values,
                zorder=0, c='grey', linestyle='--', linewidth=1)
        random_values = plotdf[(plotdf[col] == col_name) & (plotdf.algorithm.str.contains(
            "random"))].groupby(x_axis).mean().reset_index()[[x_axis, "average reward"]]
        ax.plot(random_values[x_axis].values, random_values['average reward'].values,
                zorder=0, c='red', linestyle='--', linewidth=1)
        ax.set_title(labels[label_idx].upper() +
                     "                              ")
        label_idx += 1
        ax.set_xticks([], minor=True)

h = plt.gca().get_lines()
h2 = []
for k in methods:
    h2.append(Line2D([0], [0], color=palette[k],
              dashes=linestyles[k], label=k))
h = h2 
new_hues = list(methods)

lg = ax.legend(loc='upper center', bbox_to_anchor=(-.2, 1.55+(.15 if True else .25)),
               fancybox=True, shadow=True, ncol=2, handles=h[:], labels=new_hues, handlelength=2.8, frameon=False)

plt.ylim(0, 1)
os.makedirs('figures', exist_ok=True)
plt.savefig(f'figures/config_N.pdf',
            bbox_extra_artists=(lg,), bbox_inches='tight')


In [ ]:
from matplotlib.lines import Line2D
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_context("poster")
row = None 
col = 'problem' 
style = 'correlated errors'
style = None
x_axis = 'ϵ'
yaxis = 'average reward'
ALPHA = 1


plotdf = gdf[(True)
             & alg_mask
             & (gdf['n_trials'].isin([100]))
             & (~gdf.algorithm.str.contains("expert"))
             ]

methods = [v for v in plotdf.Method.unique() if v not in ("optimal", "random")]
methods = sorted(methods, key=lambda s: (
    'Meta-CMAB' not in s, 'Expert' in s, s))
g = sns.relplot(data=plotdf[(~plotdf.algorithm.str.contains("optimal"))
                            & (~plotdf.algorithm.str.contains("random"))],
                y=yaxis, x=x_axis, hue="Method", hue_order=methods, palette=palette, ci=None, kind='line', legend=False, style='Method', dashes=linestyles, row=row, col=col)

Ks = sorted(plotdf.K.unique())
labels = "ABCDEFGH"
label_idx = 0
for ax_rows in g.axes:
    for ax, col_name in tqdm(zip(ax_rows, sorted(plotdf[col].unique())), total=len(sorted(plotdf[col].unique()))):

        random_values = plotdf[(plotdf[col] == col_name) & (plotdf.algorithm.str.contains(
            "optimal"))].groupby(x_axis).mean().reset_index()[[x_axis, "average reward"]]
        ax.plot(random_values[x_axis].values, random_values['average reward'].values,
                zorder=0, c='grey', linestyle='--', linewidth=1)
        random_values = plotdf[(plotdf[col] == col_name) & (plotdf.algorithm.str.contains(
            "random"))].groupby(x_axis).mean().reset_index()[[x_axis, "average reward"]]
        ax.plot(random_values[x_axis].values, random_values['average reward'].values,
                zorder=0, c='red', linestyle='--', linewidth=1)
        ax.set_title(labels[label_idx].upper() +
                     "                              ")
        label_idx += 1

h = plt.gca().get_lines()
h2 = []
for k in methods:
    h2.append(Line2D([0], [0], color=palette[k],
              dashes=linestyles[k], label=k))
h = h2
new_hues = list(methods)

lg = ax.legend(loc='upper center', bbox_to_anchor=(-.2, 1.45+(.15 if True else .25)),
               fancybox=True, shadow=True, ncol=3, handles=h[:], labels=new_hues, handlelength=2.8, frameon=False)

plt.ylim(0, 1)
os.makedirs('figures', exist_ok=True)
plt.savefig(f'figures/type_comparison.pdf',
            bbox_extra_artists=(lg,), bbox_inches='tight')


In [ ]:
from curlyBrace import curlyBrace
from matplotlib.lines import Line2D
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_context("poster")
row = 'problem'
col = 'spread_variance'  
style = 'correlated errors'
style = None
x_axis = 'ϵ'
yaxis = 'average reward'
ALPHA = 1


plotdf = gdf[(True)
             & alg_mask
             & (gdf['n_trials'].isin([100]))
             & (~gdf.algorithm.str.contains("expert"))

             ]
row_order = sorted(plotdf[row].unique(), reverse=True)
methods = [v for v in plotdf.Method.unique() if v not in ("optimal", "random")]
methods = sorted(methods, key=lambda s: (
    'Meta-CMAB' not in s, 'Expert' in s, s))
g = sns.relplot(data=plotdf[(~plotdf.algorithm.str.contains("optimal"))
                            & (~plotdf.algorithm.str.contains("random"))],
                y=yaxis, x=x_axis, hue="Method", hue_order=methods, row_order=row_order, palette=palette, ci=None, kind='line', legend=False, style='Method', dashes=linestyles, row=row, col=col)

Ks = sorted(plotdf.K.unique())
labels = "ABCDEFGH"
label_idx = 0
for ax_rows, row_name in zip(g.axes, row_order):
    for ax, col_name in tqdm(zip(ax_rows, sorted(plotdf[col].unique())), total=len(sorted(plotdf[col].unique()))):

        random_values = plotdf[(plotdf[col] == col_name) & (plotdf[row] == row_name) & (
            plotdf.algorithm.str.contains("optimal"))].groupby(x_axis).mean().reset_index()[[x_axis, "average reward"]]
        ax.plot(random_values[x_axis].values, random_values['average reward'].values,
                zorder=0, c='grey', linestyle='--', linewidth=1)
        random_values = plotdf[(plotdf[col] == col_name) & (plotdf[row] == row_name) & (
            plotdf.algorithm.str.contains("random"))].groupby(x_axis).mean().reset_index()[[x_axis, "average reward"]]
        ax.plot(random_values[x_axis].values, random_values['average reward'].values,
                zorder=0, c='red', linestyle='--', linewidth=1)
        ax.set_title(labels[label_idx].upper() +
                     "                              ")
        label_idx += 1

h = plt.gca().get_lines()
h2 = []
for k in methods:
    h2.append(Line2D([0], [0], color=palette[k],
              dashes=linestyles[k], label=k))
h = h2
new_hues = list(methods)

lg = ax.legend(loc='upper center', bbox_to_anchor=(-.2, 2.65+(.15 if True else .25)),
               fancybox=True, shadow=True, ncol=3, handles=h[:], labels=new_hues, handlelength=2.8, frameon=False)

plt.ylim(0, 1)
os.makedirs('figures', exist_ok=True)


k_r1 = 0.05

font = {'family': 'serif',
        'color':  'k',
        'weight': 'bold',
        'size': 20,
        }


ax2 = plt.axes([0, -0.1, 1.1, 1.1], facecolor=(1, 1, 1, 0))
ax2.set_xlim(0, 1)
ax2.set_ylim(0, 1)
curlyBrace(plt.gcf(), ax2,  [0.905, 0.55], [0.905, 0.15], k_r1, bool_auto=False,
           str_text="regression", color='r', lw=2, int_line_num=1, fontdict=font)

curlyBrace(plt.gcf(), ax2,   [0.905, .98], [0.905, .56], k_r1, bool_auto=False,
           str_text="classification", color='r', lw=2, int_line_num=1, fontdict=font)

curlyBrace(plt.gcf(), ax2,  [0.48, 0.095], [0.09, 0.095], k_r1, bool_auto=False,
           str_text="homogeneous", color='r', lw=2, int_line_num=1, fontdict=font)

curlyBrace(plt.gcf(), ax2,   [.9, 0.095], [0.49, .095], k_r1, bool_auto=False,
           str_text="heterogeneous", color='r', lw=2, int_line_num=1, fontdict=font)

ax2.spines['right'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax2.spines['left'].set_visible(False)
ax2.spines['bottom'].set_visible(False)
ax2.set_yticklabels([])
ax2.set_xticklabels([])
ax2.set_xticks([])
ax2.set_yticks([])
plt.savefig(f'figures/type_distr_comparison.pdf',
            bbox_extra_artists=(lg,), bbox_inches='tight')
